# Parallelized version

In [1]:
import math
import random
import numpy as np
import time
from functools import reduce

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("appName").setMaster("local[*]")
sc = SparkContext(conf=conf)

22/12/16 15:11:03 WARN Utils: Your hostname, Alexs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.132 instead (on interface en0)
22/12/16 15:11:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 15:11:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Functions

### Auxiliar functions

In [4]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[1:], numbers[0])
    
    return numbers

In [5]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [6]:
def parallelAssign2cluster_slower(x):
    global c
    min_distance = np.Inf
    centroid = 0
    for it, cen in enumerate(c.value):
        d = 0
        for y,z in zip(x[0],cen):
            d += abs(y-z)
        if d <= min_distance:
            min_distance = d
            centroid = it
        
    return (centroid, x[0])

In [7]:
def parallelKMeans_slower(data, K, n_iter):
    
    data_len = sc.broadcast(data.count())
    
    global c
    c = data.takeSample(0, K)
    c = [datum[0] for datum in c]
    c = sc.broadcast(c)
    
    for iter in range(n_iter):
        data_asigned = data.map(parallelAssign2cluster_slower)
        c = sc.broadcast(data_asigned.reduceByKey(lambda x,y: [q+w for q,w in zip(x,y)]) \
            .map(lambda x: [a/data_len.value for a in x[1]]).collect())
    
    return c.value

### Mandatory functions

In [8]:
def parallelReadFile(filename):
    dataset = sc.textFile(filename)
    
    # Extract the header from the data
    header = dataset.first()
    dataset = dataset.filter(lambda row: row != header)
    
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 1)

In [9]:
def parallelAssign2cluster(x):
    global c
    min_distance = np.Inf
    centroid = 0
    for it, cen in enumerate(c.value):
        d = reduce(lambda x,y:abs(x)+abs(y), [y-z for y,z in zip(x[0],cen)])
        if d <= min_distance:
            min_distance = d
            centroid = it
        
    return (centroid, x[0])

In [10]:
def parallelKMeans(data, K, n_iter):
    
    data_len = sc.broadcast(data.count())
    
    global c
    c = data.takeSample(0, K)
    c = [datum[0] for datum in c]
    c = sc.broadcast(c)
    
    for iter in range(n_iter):
        data_asigned = data.map(parallelAssign2cluster)
        len_per_cluster = sc.broadcast(data_asigned.countByKey())
        c = sc.broadcast(data_asigned.reduceByKey(lambda x,y: [q+w for q,w in zip(x,y)]) \
            .map(lambda x: [a/len_per_cluster.value[x[0]] for a in x[1]]).collect())
    
    return c.value

## Testing

In [11]:
RDD_Xy = parallelReadFile("../data/tot_mnist_shuf.csv")
RDD_Xy.count()

70000

In [12]:
centroids = parallelKMeans(RDD_Xy, 3, 2)

In [13]:
elapsed_times = []

core_list = list(range(1,9,1))
cluster_list = [3,5,7,8,9,10,11]
functions = [parallelKMeans, parallelKMeans_slower]

for cores in core_list:
    for clusters in cluster_list:
        for Kmeans in functions:
            global sc
            sc.stop()
            conf = SparkConf().setAppName("appName").setMaster(f"local[{cores}]")
            sc = SparkContext(conf=conf)

            print(f"---------- Starting execution with {cores} cores and {clusters} clusters ----------")

            start = time.time()

            # ---------- Execution ----------

            RDD_Xy = parallelReadFile("../data/tot_mnist_shuf.csv")
            Kmeans(RDD_Xy, clusters, 3)

            # ---------- Execution ----------

            end = time.time()
            print(f"---------- Finished execution with {cores} cores and {clusters} clusters ----------")
            elapsed_time = end - start
            elapsed_times.append(
                {
                    "Number of Cores": cores,
                    "Number of Clusters": clusters,
                    "Function used": Kmeans,
                    "elapsed_time": elapsed_time
                }
            )
            print(f"Elapsed time for cores {cores} is {elapsed_time} seconds")

---------- Starting execution with 1 cores and 3 clusters ----------


---------- Finished execution with 1 cores and 3 clusters ----------
Elapsed time for cores 1 is 279.1488049030304 seconds
---------- Starting execution with 1 cores and 3 clusters ----------


---------- Finished execution with 1 cores and 3 clusters ----------
Elapsed time for cores 1 is 112.05107283592224 seconds
---------- Starting execution with 1 cores and 5 clusters ----------


---------- Finished execution with 1 cores and 5 clusters ----------
Elapsed time for cores 1 is 429.0091631412506 seconds
---------- Starting execution with 1 cores and 5 clusters ----------


---------- Finished execution with 1 cores and 5 clusters ----------
Elapsed time for cores 1 is 148.6441731452942 seconds
---------- Starting execution with 1 cores and 7 clusters ----------


...

22/12/17 02:27:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 906913 ms exceeds timeout 120000 ms
22/12/17 02:27:25 WARN SparkContext: Killing executors is not supported by current scheduler.


---------- Finished execution with 8 cores and 10 clusters ----------
Elapsed time for cores 8 is 4965.233966827393 seconds
---------- Starting execution with 8 cores and 10 clusters ----------


---------- Finished execution with 8 cores and 10 clusters ----------
Elapsed time for cores 8 is 1040.8048272132874 seconds
---------- Starting execution with 8 cores and 11 clusters ----------


---------- Finished execution with 8 cores and 11 clusters ----------
Elapsed time for cores 8 is 4871.529038906097 seconds
---------- Starting execution with 8 cores and 11 clusters ----------


---------- Finished execution with 8 cores and 11 clusters ----------
Elapsed time for cores 8 is 1996.0898070335388 seconds


In [14]:
elapsed_times

[{'Number of Cores': 1,
  'Number of Clusters': 3,
  'Function used': <function __main__.parallelKMeans(data, K, n_iter)>,
  'elapsed_time': 279.1488049030304},
 {'Number of Cores': 1,
  'Number of Clusters': 3,
  'Function used': <function __main__.parallelKMeans_slower(data, K, n_iter)>,
  'elapsed_time': 112.05107283592224},
 {'Number of Cores': 1,
  'Number of Clusters': 5,
  'Function used': <function __main__.parallelKMeans(data, K, n_iter)>,
  'elapsed_time': 429.0091631412506},
 {'Number of Cores': 1,
  'Number of Clusters': 5,
  'Function used': <function __main__.parallelKMeans_slower(data, K, n_iter)>,
  'elapsed_time': 148.6441731452942},
 {'Number of Cores': 1,
  'Number of Clusters': 7,
  'Function used': <function __main__.parallelKMeans(data, K, n_iter)>,
  'elapsed_time': 569.6051461696625},
 {'Number of Cores': 1,
  'Number of Clusters': 7,
  'Function used': <function __main__.parallelKMeans_slower(data, K, n_iter)>,
  'elapsed_time': 170.84679698944092},
 {'Number o